# TFLite Interpreter

This code is based on `rl_evaluate_policy` package `print_returns_positions module`.

In [2]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
!rm -rf msca-race-simulation/
!git clone --depth 1 https://github.com/pezon/msca-race-simulation 
!cp -R msca-race-simulation/* .

Cloning into 'msca-race-simulation'...
remote: Enumerating objects: 365, done.
remote: Counting objects: 100% (365/365), done.
remote: Compressing objects: 100% (304/304), done.
remote: Total 365 (delta 175), reused 209 (delta 52), pack-reused 0
Receiving objects: 100% (365/365), 4.15 MiB | 18.62 MiB/s, done.
Resolving deltas: 100% (175/175), done.


In [4]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.4/624.4 kB 46.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 98.7 MB/s eta 0:00:00
  Created wheel for gym: filename=gym-0.23.0-py3-none-any.whl size=697642 sha256=3397603f5caeac76e0cf004f270803ba7f7d2b0c835f1b73ac1ee11fc978e761
  Stored in directory: /root/.cache/pip/wheels/3d/6f/b4/3991d4fae11d0ecb0754c11cc1b4e7745012850da4efaaf0b1
Successfully built gym
  Attempting uninstall: tensorflow-probability
    Found existing installation: tensorflow-probability 0.20.1
    Uninstalling tensorflow-probability-0.20.1:
      Succes

In [6]:
import tensorflow as tf
from tf_agents.trajectories.time_step import TimeStep

from machine_learning_rl_training.src.rl_environment_single_agent import RaceSimulation
from machine_learning_rl_training.src.rl_evaluate_policy import convert_time_step, print_returns_positions

model_path = "/content/gdrive/MyDrive/RLF002/vse-002/exports/2023-05-21-final/nn_reinforcement_Shanghai_2019.tflite"
race_pars_file = "/content/racesim/input/parameters/pars_Shanghai_2019.ini"
mcs_pars_file = "/content/racesim/input/parameters/pars_mcs.ini"
vse_others = "basestrategy"

vse_paths = {
    "supervised_nnmodel_cc": "/content/racesim/input/vse/nn_supervised_compoundchoice.tflite",
    "supervised_nnmodel_tc": "/content/racesim/input/vse/nn_supervised_tirechange.tflite",
    "supervised_preprocessor_cc": "/content/racesim/input/vse/preprocessor_supervised_compoundchoice.pkl",
    "supervised_preprocessor_tc": "/content/racesim/input/vse/preprocessor_supervised_tirechange.pkl"
}

py_env = RaceSimulation(
    race_pars_file=race_pars_file,
    mcs_pars_file=mcs_pars_file,
    vse_type=vse_others,
    use_prob_infl=True,
    create_rand_events=True,
    vse_paths=vse_paths)

print(
  py_env.action_spec(),
  py_env.observation_spec(),
)

interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()
# input_index = interpreter.get_input_details()[0]["index"]  # old model
input_index = interpreter.get_input_details()[2]["index"]  # new model
output_index = interpreter.get_output_details()[0]["index"]

BoundedArraySpec(shape=(), dtype=dtype('int32'), name='action', minimum=0, maximum=3) BoundedArraySpec(shape=(40,), dtype=dtype('float32'), name='observation', minimum=0.0, maximum=1.0)


In [8]:
time_step = py_env.reset()
episode_return = 0.0

lap = 1
while not time_step.is_last():
    converted_time_step = convert_time_step(time_step)
    observation = converted_time_step.observation
    interpreter.set_tensor(2, converted_time_step.observation)
    interpreter.invoke()
    print(interpreter.get_tensor(output_index))
    # action_q = interpreter.get_tensor(output_index)[0].argmax()  # old model
    action_q = interpreter.get_tensor(output_index)  # new model
    print(f"driver = {py_env.idx_driver}, lap = {lap}, action = {action_q}")
    time_step = py_env.step(action_q[0])
    episode_return += time_step.reward
    lap += 1

final_position = py_env.race.positions[py_env.race.get_last_compl_lap(py_env.idx_driver), py_env.idx_driver]
print(f"driver = {py_env.idx_driver}, final_position = {final_position}")

[0]
driver = 12, lap = 1, action = [0]
[0]
driver = 12, lap = 2, action = [0]
[0]
driver = 12, lap = 3, action = [0]
[0]
driver = 12, lap = 4, action = [0]
[0]
driver = 12, lap = 5, action = [0]
[0]
driver = 12, lap = 6, action = [0]
[0]
driver = 12, lap = 7, action = [0]
[0]
driver = 12, lap = 8, action = [0]
[0]
driver = 12, lap = 9, action = [0]
[0]
driver = 12, lap = 10, action = [0]
[0]
driver = 12, lap = 11, action = [0]
[0]
driver = 12, lap = 12, action = [0]
[0]
driver = 12, lap = 13, action = [0]
[0]
driver = 12, lap = 14, action = [0]
[0]
driver = 12, lap = 15, action = [0]
[0]
driver = 12, lap = 16, action = [0]
[0]
driver = 12, lap = 17, action = [0]
[0]
driver = 12, lap = 18, action = [0]
[0]
driver = 12, lap = 19, action = [0]
[0]
driver = 12, lap = 20, action = [0]
[0]
driver = 12, lap = 21, action = [0]
[0]
driver = 12, lap = 22, action = [0]
[0]
driver = 12, lap = 23, action = [0]
[0]
driver = 12, lap = 24, action = [0]
[0]
driver = 12, lap = 25, action = [0]
[0]
drive

In [9]:
interpreter.get_tensor(output_index)

array([0], dtype=int32)